In [1]:
!pip install fastapi==0.112.4 uvicorn==0.25.0 sqlalchemy==2.0.35 sqlmodel==0.0.24 sqlalchemy-libsql==0.1.0 libsql-experimental==0.0.45 -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.9/93.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.3/60.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 4.3 MB/s eta 0:00:00


In [2]:
TURSO_DATABASE_URL=""
TURSO_AUTH_TOKEN=""

In [ ]:
import libsql_experimental as libsql
from sqlmodel import Field, Session, SQLModel, create_engine, select, Column, JSON
from typing import Optional, List, Dict, Any, Union, Literal
import time

url = TURSO_DATABASE_URL
auth_token = TURSO_AUTH_TOKEN
DB_FILE = "codex-ai-news-reports.db"

conn = libsql.connect(DB_FILE, sync_url=url, auth_token=auth_token)
conn.sync()


# SQLModel用のエンジン設定
sqlite_url = f"sqlite:///{DB_FILE}"
connect_args = {"check_same_thread": False}
engine = create_engine(sqlite_url, echo=True, connect_args=connect_args)
print("done!")

In [ ]:
# 基本のResearchTaskモデルを定義
class ResearchTask(SQLModel, table=True):
    """リサーチタスク情報を保存するモデル"""
    task_id: str = Field(primary_key=True)
    query: str
    status: str
    created_at: float
    result: Optional[Dict[str, Any]] = Field(default=None, sa_column=Column(JSON))
    error: Optional[str] = None
    log_file: Optional[str] = None
    output_dir: Optional[str] = None
    running_message: Optional[str] = None
    progress: Optional[int] = None

def create_db_and_tables():
    """データベースとテーブルを作成する関数"""
    print("データベースとテーブルを作成します")
    SQLModel.metadata.create_all(engine)
    conn.commit()
    conn.sync()

def on_startup():
    """アプリケーション起動時の初期化処理"""
    print("アプリケーション起動時の初期化処理を実行します")
    create_db_and_tables()

on_startup()
print("基本テーブルを作成しました")

In [ ]:
# レポート構造に対応するデータモデルを追加
from datetime import datetime
import json
import uuid

class ReportSection(SQLModel, table=True):
    """レポートの各セクション情報を保存するモデル"""
    section_id: str = Field(primary_key=True)
    task_id: str = Field(foreign_key="researchtask.task_id")
    section_type: str  # executive_summary, key_findings, deep_dive, risks, recommendations, references
    title: str
    content: str
    section_metadata: Optional[Dict[str, Any]] = Field(default=None, sa_column=Column(JSON))
    created_at: float
    order_index: int = 0

class ReportData(SQLModel, table=True):
    """完全なレポートデータを保存するモデル"""
    report_id: str = Field(primary_key=True)
    task_id: str = Field(foreign_key="researchtask.task_id")
    topic: str
    report_date: str
    audience: Optional[str] = None
    executive_summary: str
    key_findings: List[Dict[str, Any]] = Field(default_factory=list, sa_column=Column(JSON))
    deep_dive_sections: List[Dict[str, Any]] = Field(default_factory=list, sa_column=Column(JSON))
    risks_issues: List[str] = Field(default_factory=list, sa_column=Column(JSON))
    recommendations: Dict[str, List[Dict[str, Any]]] = Field(default_factory=dict, sa_column=Column(JSON))
    references: List[Dict[str, Any]] = Field(default_factory=list, sa_column=Column(JSON))
    markdown_content: Optional[str] = None
    created_at: float
    updated_at: Optional[float] = None

# 新しいテーブルを作成
SQLModel.metadata.create_all(engine)
conn.commit()
conn.sync()
print("レポート用テーブルを作成しました")

In [ ]:
# レポート生成用のJSONからMarkdownへの変換機能
def generate_markdown_report(report_data: ReportData) -> str:
    """レポートデータからMarkdownを生成する関数"""
    
    # タイトルと日付
    markdown = f"# {report_data.topic} / {report_data.report_date}\n\n"
    
    # Executive Summary
    markdown += "## Executive Summary\n\n"
    markdown += f"{report_data.executive_summary}\n\n"
    
    # Key Findings
    markdown += "## Key Findings\n\n"
    for i, finding in enumerate(report_data.key_findings, 1):
        finding_text = finding.get('text', '')
        reference = finding.get('reference', '')
        if reference:
            markdown += f"{i}. {finding_text} [{reference}]\n"
        else:
            markdown += f"{i}. {finding_text}\n"
    markdown += "\n"
    
    # Deep Dive
    if report_data.deep_dive_sections:
        markdown += "## Deep Dive\n\n"
        for section in report_data.deep_dive_sections:
            title = section.get('title', '')
            content = section.get('content', '')
            figures = section.get('figures', [])
            
            markdown += f"### {title}\n\n"
            markdown += f"{content}\n\n"
            
            # 図表の参照を追加
            for fig in figures:
                fig_path = fig.get('path', '')
                fig_caption = fig.get('caption', '')
                if fig_path:
                    markdown += f"![{fig_caption}]({fig_path})\n\n"
    
    # Risks & Open Issues
    if report_data.risks_issues:
        markdown += "## Risks & Open Issues\n\n"
        for risk in report_data.risks_issues:
            markdown += f"- {risk}\n"
        markdown += "\n"
    
    # Recommendations
    if report_data.recommendations:
        markdown += "## Recommendations\n\n"
        for audience, recs in report_data.recommendations.items():
            markdown += f"### {audience}\n\n"
            for rec in recs:
                action = rec.get('action', '')
                priority = rec.get('priority', '')
                cost = rec.get('cost', '')
                markdown += f"- **{action}** (優先度: {priority}, コスト: {cost})\n"
            markdown += "\n"
    
    # References
    if report_data.references:
        markdown += "## References\n\n"
        for i, ref in enumerate(report_data.references, 1):
            url = ref.get('url', '')
            title = ref.get('title', '')
            published = ref.get('published', '')
            accessed = ref.get('accessed', '')
            markdown += f"{i}. [{title}]({url}) (発行日: {published}, アクセス日: {accessed})\n"
    
    return markdown

# レポートデータをTurso DBに保存する関数
def save_report_to_db(report_data: ReportData) -> str:
    """レポートデータをデータベースに保存する関数"""
    
    # Markdownを生成（セッション外で実行）
    markdown_content = generate_markdown_report(report_data)
    report_data.markdown_content = markdown_content
    report_data.updated_at = time.time()
    
    with Session(engine) as session:
        # 既存のレポートがあるかチェック
        existing_report = session.get(ReportData, report_data.report_id)
        
        if existing_report:
            # 既存レポートを更新
            existing_report.topic = report_data.topic
            existing_report.report_date = report_data.report_date
            existing_report.audience = report_data.audience
            existing_report.executive_summary = report_data.executive_summary
            existing_report.key_findings = report_data.key_findings
            existing_report.deep_dive_sections = report_data.deep_dive_sections
            existing_report.risks_issues = report_data.risks_issues
            existing_report.recommendations = report_data.recommendations
            existing_report.references = report_data.references
            existing_report.markdown_content = markdown_content
            existing_report.updated_at = time.time()
            session.add(existing_report)
        else:
            # 新しいレポートを追加
            session.add(report_data)
        
        session.commit()
        session.refresh(report_data if not existing_report else existing_report)
    
    # データベース同期
    conn.sync()
    
    return markdown_content

print("レポート生成機能を実装しました")

In [ ]:
# サンプルレポートデータの作成とテスト
def create_sample_report_data() -> ReportData:
    """サンプルレポートデータを作成する関数"""
    
    # 現在の日付を取得
    current_date = datetime.now().strftime("%Y-%m-%d")
    report_id = f"report-{uuid.uuid4().hex[:8]}"
    
    # Key Findings のサンプル構造
    key_findings = [
        {
            "text": "Turso DBは分散SQLiteデータベースとして高いパフォーマンスを提供",
            "reference": "1",
            "confidence": "high"
        },
        {
            "text": "libsql-experimentalライブラリによりPythonから簡単にアクセス可能",
            "reference": "2",
            "confidence": "high"
        },
        {
            "text": "同期機能により複数のレプリカ間でデータの整合性を保持",
            "reference": "3",
            "confidence": "medium"
        },
        {
            "text": "SQLModelとの統合により型安全なデータベース操作が可能",
            "reference": "4",
            "confidence": "high"
        },
        {
            "text": "クラウドとローカルの両方でシームレスな動作を実現",
            "reference": "5",
            "confidence": "medium"
        }
    ]
    
    # Deep Dive セクションのサンプル構造
    deep_dive_sections = [
        {
            "title": "Turso DB アーキテクチャ",
            "content": "Turso DBは分散SQLiteアーキテクチャを採用し、複数のレプリカ間でデータの同期を行います。libsql-experimentalライブラリを使用することで、Pythonアプリケーションから直接アクセスが可能になります。",
            "figures": [
                {
                    "path": "../figs/turso-architecture.png",
                    "caption": "Turso DB アーキテクチャ図"
                }
            ]
        },
        {
            "title": "パフォーマンス特性",
            "content": "ベンチマークテストの結果、Turso DBは従来のPostgreSQLと比較して読み取り操作で約30%の性能向上を示しました。特に分散環境での同期処理において優れた性能を発揮します。",
            "figures": [
                {
                    "path": "../figs/performance-comparison.png",
                    "caption": "パフォーマンス比較グラフ"
                }
            ]
        }
    ]
    
    # Risks & Issues のサンプル
    risks_issues = [
        "実験的ライブラリ（libsql-experimental）のため、本番環境での安定性に不確実性がある",
        "ドキュメンテーションが限定的で、トラブルシューティングが困難な場合がある",
        "大規模なデータセットでの性能特性が未検証",
        "エコシステムの成熟度が他のデータベースソリューションと比較して低い"
    ]
    
    # Recommendations のサンプル構造
    recommendations = {
        "開発者": [
            {
                "action": "小規模なプロトタイプでTurso DBの動作を検証する",
                "priority": "高",
                "cost": "低（1-2日）"
            },
            {
                "action": "既存のSQLiteアプリケーションからの移行パスを調査する",
                "priority": "中",
                "cost": "中（1週間）"
            }
        ],
        "プロダクトマネージャー": [
            {
                "action": "ビジネス要件とTurso DBの機能マッチングを評価する",
                "priority": "高",
                "cost": "中（3-5日）"
            },
            {
                "action": "競合データベースソリューションとのコスト比較を実施する",
                "priority": "中",
                "cost": "中（1週間）"
            }
        ],
        "DevOpsエンジニア": [
            {
                "action": "Turso DBの監視・運用手順を策定する",
                "priority": "中",
                "cost": "高（2-3週間）"
            }
        ]
    }
    
    # References のサンプル構造
    references = [
        {
            "url": "https://turso.tech/docs",
            "title": "Turso Database Documentation",
            "published": "2024-09-20",
            "accessed": current_date
        },
        {
            "url": "https://github.com/tursodatabase/libsql-experimental-python",
            "title": "libsql-experimental Python Library",
            "published": "2024-09-15",
            "accessed": current_date
        },
        {
            "url": "https://blog.turso.tech/introducing-turso-db",
            "title": "Introducing Turso: The Edge Database",
            "published": "2024-08-30",
            "accessed": current_date
        },
        {
            "url": "https://sqlmodel.tiangolo.com/",
            "title": "SQLModel Documentation",
            "published": "2024-09-10",
            "accessed": current_date
        },
        {
            "url": "https://benchmarks.turso.tech/",
            "title": "Turso Performance Benchmarks",
            "published": "2024-09-25",
            "accessed": current_date
        }
    ]
    
    # ReportData インスタンスを作成
    report_data = ReportData(
        report_id=report_id,
        task_id="test-18675145",
        topic="Turso DB と Python 統合の技術調査",
        report_date=current_date,
        audience="開発チーム",
        executive_summary="Turso DBは分散SQLiteデータベースとして、Pythonアプリケーションとの統合において優れた性能と柔軟性を提供します。libsql-experimentalライブラリを使用することで、型安全なデータベース操作が可能になり、クラウドとローカル環境でのシームレスな動作を実現できます。ただし、実験的な性質により本番環境での採用には慎重な検討が必要です。",
        key_findings=key_findings,
        deep_dive_sections=deep_dive_sections,
        risks_issues=risks_issues,
        recommendations=recommendations,
        references=references,
        created_at=time.time()
    )
    
    return report_data

# サンプルデータを作成してテスト
sample_report = create_sample_report_data()
print("サンプルレポートデータを作成しました")

In [ ]:
# レポートデータをTurso DBに保存してMarkdownを生成するテスト
def test_report_generation():
    """レポート生成機能のテスト"""
    
    # サンプルレポートデータを作成
    sample_report = create_sample_report_data()
    
    # データベースに保存してMarkdownを生成
    try:
        markdown_content = save_report_to_db(sample_report)
        
        print("=== 生成されたMarkdownレポート ===")
        print(markdown_content[:500] + "..." if len(markdown_content) > 500 else markdown_content)
        print("\n=== データベースへの保存完了 ===")
        print(f"レポートID: {sample_report.report_id}")
        print(f"タスクID: {sample_report.task_id}")
        print(f"トピック: {sample_report.topic}")
        print(f"作成日時: {datetime.fromtimestamp(sample_report.created_at)}")
        
        # データベースから読み取りテスト
        saved_report = get_report_from_db(sample_report.report_id)
        if saved_report:
            print(f"データベース読み取りテスト: 成功 (ID: {saved_report.report_id})")
        else:
            print("データベース読み取りテスト: 失敗")
        
        return sample_report, markdown_content
        
    except Exception as e:
        print(f"テスト実行中にエラーが発生しました: {e}")
        import traceback
        traceback.print_exc()
        return None, None

# データベースからレポートを取得する関数
def get_report_from_db(report_id: str) -> Optional[ReportData]:
    """データベースからレポートを取得する関数"""
    with Session(engine) as session:
        statement = select(ReportData).where(ReportData.report_id == report_id)
        result = session.exec(statement).first()
        if result:
            # セッション外でアクセスできるように属性を明示的に読み込み
            _ = result.report_id
            _ = result.topic
            _ = result.executive_summary
            _ = result.key_findings
            _ = result.deep_dive_sections
            _ = result.risks_issues
            _ = result.recommendations
            _ = result.references
            _ = result.markdown_content
        return result

# レポート一覧を取得する関数
def list_reports_from_db() -> List[ReportData]:
    """データベースからすべてのレポートを取得する関数"""
    with Session(engine) as session:
        statement = select(ReportData)
        results = session.exec(statement).all()
        # セッション外でアクセスできるように属性を明示的に読み込み
        for result in results:
            _ = result.report_id
            _ = result.topic
            _ = result.executive_summary
            _ = result.key_findings
            _ = result.deep_dive_sections
            _ = result.risks_issues
            _ = result.recommendations
            _ = result.references
            _ = result.markdown_content
        return list(results)

print("データベース操作用の関数を定義しました")

# レポート生成のテスト実行
try:
    test_report, markdown = test_report_generation()
    print("\\nレポート生成テストが正常に完了しました！")
except Exception as e:
    print(f"エラーが発生しました: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
# 全体のテストとデモンストレーション
print("=== Codex AI News Reports システムのテスト ===")
print("1. データベース接続とテーブル作成: 完了")
print("2. レポートデータモデル: 完了") 
print("3. Markdown生成機能: 完了")
print("4. Turso DB同期機能: 完了")
print("\n実行可能な関数:")
print("- create_sample_report_data(): サンプルレポート作成")
print("- save_report_to_db(report_data): レポートをDBに保存")
print("- generate_markdown_report(report_data): Markdown生成")
print("- get_report_from_db(report_id): レポート取得")
print("- list_reports_from_db(): 全レポート一覧")
print("\nテスト実行: 上のセルで自動的に test_report_generation() が実行されます")